Небольшой проект по парсингу сайта на основе реальной задачи

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import csv
import numpy as np

In [ ]:
URL = '...dpfrom=01.01.2019&dpto=29.04.2020&status=197%3B191%3B201%3B210%3B318%3B190%3B192%3B193%3B194%3B195%3B196%3B198%3B200%3B202%3B203%3B208%3B204%3B317%3B205%3B206%3B207%3B233%3B234%3B209&limit=500'
HEADERS = {'user-agent':UserAgent().chrome,
          'accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'}
HEADERS1 = {'user-agent':UserAgent().chrome,
          'accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'}
HOST = '....'

In [ ]:
def get_html(url,params=None):
    r = requests.get(url,headers=HEADERS,params=params)
    return r

In [ ]:
def get_content(html):
    soup = BeautifulSoup(html,'html.parser')
    items = soup.find_all('div',class_="section-procurement__item")

    lots = []
    for item in items:
        www = item.find('a',class_="section-procurement__item-title").get('href')
        pp = HOST+www

        req_get = requests.get(pp,headers=HEADERS1)
        zz = req_get.text
        soup1 = BeautifulSoup(zz,'html.parser')
        items1 = soup1.find('div',class_="procedure__lot-item").find_next("td").find_next("td").get_text(strip=True)
        
        items2 = soup1.find(text='Заказчик:').find_next("td").get_text(strip=True)

        xx.append({'Номер процедуры':
                   item.find('div',class_="section-procurement__item-numbers")\
                                                       .get_text(strip=True)[-9:],
                  'Наименование_процедуры': 
                   item.find('a',class_="section-procurement__item-title").get_text(strip=True),
                  'Наименование_организатора':
                   item.find_all('div',class_="section-procurement__item-description")\
                                                                       [0].find('a')\
                   .get_text(strip=True),
                  'Регионы':
                   p_i(item),
                   'Тип_продаваемого_имущества':
                   t_i(item),
                   'Дата_публикации':
                   item.find('div',class_="section-procurement__item-dateTo")\
                                                       .get_text(strip=True)[16:],
                   'Статус':
                   st_ui(item),
                   'Начальная_цена':
                   c_i(item),
                   'Предмет_договора':
                   items1,
                   'Заказчик':
                   items2
                   
                  })

    return lots

In [ ]:
def st_ui(item):
    '''Функция для парсинга статуса торговой площадки'''
    try:
        return item.find('span',class_="section-procurement__item-dateTo").get_text(strip=True)[7:]
    except AttributeError:
        return 'Отстутствует'

In [ ]:
def p_i(item):
    '''Функция для парсинга региона торговой площадки'''
    try:
        return item.find(text='Регионы:').find_next("a").get_text(strip=True)
    except AttributeError:
        return 'Отстутствует'

In [ ]:
def t_i(item):
    '''Функция для парсинга типа реаизуемого имущества'''
    try:
        return item.find(text='Тип продаваемого имущества:').find_next("a").get_text(strip=True)
    except AttributeError:
        return 'Отстутствует'

In [ ]:
def  c_i(item):
    '''Функция для парсинга региона торговой площадки'''
    try:
        return item.find('div',class_="section-procurement__item-totalPrice").get_text(strip=True)
    except AttributeError:
        return 'Не указана'

In [ ]:
def get_page_count(html):
    '''Функция для подсчета количества страниц с необходимыми лотами'''
    soup = BeautifulSoup(html,'html.parser')
    pagenum = soup.find('li',class_="pagination__next")
    if pagenum:
        return int(pagenum.find_previous_siblings("li")[0].get_text())
    else:
        return 1     

In [ ]:
def save_file(items,path):
    '''Функция для записи в csv'''
    with open (path,'w',newline='') as file:
        writer = csv.writer(file,delimiter=';')
        writer.writerow(['Номер процедуры', 'Наименование_процедуры', 'Наименование_организатора', 
                         'Регионы', 'Тип_продаваемого_имущества', 'Дата_публикации',
                         'Статус', 'Начальная_цена', 'Предмет_договора', 'Заказчик'])
        for item in items:
            writer.writerow([item['Номер процедуры'], item['Наименование_процедуры'], 
                                 item['Наименование_организатора'],item['Регионы_ПИ'], 
                                 item['Тип_продаваемого_имущества'], item['Дата_публикации'],
                                 item['Статус'], item['Начальная_цена'], 
                                 item['Предмет_договора'], item['Заказчик']])
        

In [ ]:
def parse():
    html = get_html(URL)
    if html.status_code == 200:
        lots = []
        
        page_count = get_page_count(html.text)
        for num in range(1,page_count+1):
            print(num)
            html = get_html(URL,params={'page':num})
            lots.extend(get_content(html.text))
            print(f'страница {num} из {page_count}')
        print(len(lots))
        save_file(lots,'lots.csv')       
    else:
        print('Error')

In [543]:
parse()

1
страница 1 из 5
2
страница 2 из 5
3
страница 3 из 5
4
страница 4 из 5
5
страница 5 из 5
2144
